In [3]:
#Import Necessary libraries
library('tidyverse')
library('dplyr')
library('forcats')
library(quantregForest)
library('kableExtra')
source('functions.R')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.1     ✔ purrr   0.3.5
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.2.1     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: randomForest

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine


The following object is masked from ‘package:ggplot2’:

    margin


Loading required package: RColorBrewer


Attaching package: ‘kableExtra’


The following object is masked from ‘package:dplyr’:

    group_rows




In [4]:
kFoldForest = function(Kfold, seed, datafr)
{ set.seed(seed)
  n = nrow(datafr)
  iperm<<-sample(n) # set as global for debugging check
  nhold = round(n/Kfold)
  reg = list()
  pred = list() 
  scoreVar = list()
  rocVar = list()
  pred_y = sample(n-nhold)
  for(k in 1:Kfold){ 
        ilow = (k-1)*nhold+1
        ihigh = k*nhold
        if(k==Kfold) { ihigh = n }
        ifold = iperm[ilow:ihigh]
      
        holdo = datafr[ifold,]
        train = datafr[-ifold,]
        train$price = log(train$price)
      
        qRF = quantregForest(train[,-1],train[,1],ntree = 100)
        predRF = predict(qRF, what=c(.1,.25,.5,.75,.9), newdata=holdo[,-1])
        preds50 = cbind(exp(predRF[,3]),exp(predRF[,2]),exp(predRF[,4]))
        preds80 = cbind(exp(predRF[,3]),exp(predRF[,1]),exp(predRF[,5]))
      
        IS50qRF = intervalScore(preds50,holdo$price,0.5)
        IS80qRF = intervalScore(preds80,holdo$price,0.8)
          
        outqRF = rbind(IS50qRF$summary,IS80qRF$summary) 
        colnames(outqRF) = c("level","avgleng","IS","cover") 
        print(outqRF)
        
  }
}

In [8]:
train = readRDS('04a-wrangledTrain.rds')
training = train %>% select(price,condition,fuel,odometer,transmission,drive,isLuxury,age)

In [9]:
kFoldForest(Kfold = 3,seed = 123,datafr = training)

     level   avgleng       IS     cover
[1,]   0.5  8299.935 15129.47 0.5375340
[2,]   0.8 16227.266 21949.24 0.8357915
     level   avgleng       IS     cover
[1,]   0.5  8300.686 15073.70 0.5402716
[2,]   0.8 16173.385 21856.36 0.8343920
     level   avgleng       IS     cover
[1,]   0.5  8379.141 15239.59 0.5349113
[2,]   0.8 16295.097 22242.79 0.8309034


In [5]:
sub = feature_selection(train) %>% select(-manufacturer)
kFoldForest(Kfold = 3, seed = 123, datafr = sub)

     level  avgleng       IS     cover
[1,]   0.5 11640.81 18918.67 0.6142169
[2,]   0.8 24329.37 30063.24 0.8938497
     level  avgleng       IS     cover
[1,]   0.5 10806.70 18776.92 0.5865951
[2,]   0.8 22137.93 28887.16 0.8755171
     level  avgleng       IS     cover
[1,]   0.5 11388.50 18986.80 0.6042724
[2,]   0.8 23381.18 29916.05 0.8791640


In [14]:
sub$price = log(sub$price)
qRF = quantregForest(sub[,-1],sub[,1],ntree = 100)

In [16]:
holdo = readRDS('04b-wrangledHoldout.rds')

In [17]:
predRF = predict(qRF, what=c(.1,.25,.5,.75,.9), newdata=holdo[,-1])

In [19]:
preds50 = cbind(exp(predRF[,3]),exp(predRF[,2]),exp(predRF[,4]))
preds80 = cbind(exp(predRF[,3]),exp(predRF[,1]),exp(predRF[,5]))     
IS50qRF = intervalScore(preds50,holdo$price,0.5)
IS80qRF = intervalScore(preds80,holdo$price,0.8)
outqRF = rbind(IS50qRF$summary,IS80qRF$summary) 
colnames(outqRF) = c("level","avgleng","IS","cover") 
print(outqRF)

     level  avgleng       IS     cover
[1,]   0.5 11346.96 18694.62 0.6035763
[2,]   0.8 23133.01 29568.49 0.8753025
